In [3]:
import re
import pandas as pd

from gtts import gTTS, lang
from playsound import playsound 

import numpy as np

#transformer imports
from fairseq.checkpoint_utils import load_model_ensemble_and_task_from_hf_hub
from fairseq.models.text_to_speech.hub_interface import TTSHubInterface
import IPython.display as ipd

#writing wav files imports
from scipy.io.wavfile import write

#silence removal libraries
from pydub import AudioSegment
from pydub.silence import detect_silence

playsound is relying on another python subprocess. Please use `pip install pygobject` if you want playsound to run more efficiently.


In [4]:
WORDS_DATA_DIR = 'data/words.csv'

words_data = pd.read_csv(WORDS_DATA_DIR, encoding='utf-8')

In [5]:
def remove_word_enclosed_in_parenthesis_from_string(text_str: str) -> str:

    return re.sub(r'[\(\[].*?[\)\]]', '', text_str)

remove_word_enclosed_in_parenthesis_from_string('آلية (تركيبة تشغيل)')

'آلية '

In [6]:
words_data['keyword_free_punctuation_clean'] = words_data['keyword_free_punctuation'].apply(lambda x: remove_word_enclosed_in_parenthesis_from_string(x))

In [8]:
words_data.head()

,id,keyword,keyword_free_punctuation,keyword_free_punctuation_clean
0,5126544,اِئْتكال,ائتكال,ائتكال
1,5126545,اِئْتِلاف,ائتلاف,ائتلاف
2,5126546,اِئْتلَفَ,ائتلف,ائتلف
3,5126547,اِئْتمان,ائتمان,ائتمان
4,5126548,اِئْتمانيّ,ائتماني,ائتماني


In [9]:
words_data = words_data.astype({'keyword': 'string',
                                'keyword_free_punctuation': 'string',
                                'keyword_free_punctuation_clean': 'string'})


    Google TTS Trial

In [10]:
def get_speech_of_text(text_string: str):
    tts = gTTS(text=text_string, lang='ar')

    tts.save(f'{text_string}.mp3')

get_speech_of_text('ائتلاف')

    Facebook tts transformer cv7 trial

In [13]:
models, cfg, task = load_model_ensemble_and_task_from_hf_hub(
    "facebook/tts_transformer-ar-cv7",
    arg_overrides={"vocoder": "hifigan", "fp16":False}
)
model = models[0]
TTSHubInterface.update_cfg_with_data_cfg(cfg, task.data_cfg)
generator = task.build_generator([model], cfg)


Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

2022-11-14 14:00:29 | INFO | fairseq.tasks.speech_to_text | dictionary size (spm_char.txt): 90
2022-11-14 14:00:30 | INFO | fairseq.models.text_to_speech.vocoder | loaded HiFiGAN checkpoint from /home/elearning/.cache/fairseq/models--facebook--tts_transformer-ar-cv7/snapshots/c6e31971876a13c2ce0e1d26097093c0ece23e20/hifigan.bin
2022-11-14 14:00:31 | INFO | fairseq.models.text_to_speech.vocoder | loaded HiFiGAN checkpoint from /home/elearning/.cache/fairseq/models--facebook--tts_transformer-ar-cv7/snapshots/c6e31971876a13c2ce0e1d26097093c0ece23e20/hifigan.bin


In [55]:
text = "ذهب الولد إلى المدرسة مع الاصدقاء و استطاعوا التسلل خلف المدرس لسرقة اسئلة الامتحان "
#text = "ذهب الولد"
sample = TTSHubInterface.get_model_input(task, text)
wav, rate = TTSHubInterface.get_prediction(task, model, generator, sample)
ipd.Audio(wav, rate=rate)

2022-11-14 14:34:52 | WARNING | phonemizer | words count mismatch on 100.0% of the lines (1/1)


In [27]:
data = np.array(wav)

In [28]:
write(filename='ذهب.wav', rate=rate, data=data)

In [29]:
sound = AudioSegment.from_file('ذهب.wav', format='wav')

In [30]:
ranges = detect_silence(sound,
                        silence_thresh=-80)                    

In [31]:
def detect_leading_silence(sound, silence_threshold=-50.0, chunk_size=10):
    '''
    sound is a pydub.AudioSegment
    silence_threshold in dB
    chunk_size in ms

    iterate over chunks until you find the first one with sound
    '''
    trim_ms = 0 # ms

    assert chunk_size > 0 # to avoid infinite loop
    while sound[trim_ms:trim_ms+chunk_size].dBFS < silence_threshold and trim_ms < len(sound):
        trim_ms += chunk_size

    return trim_ms

In [32]:
sound = AudioSegment.from_file("ذهب.wav", format="wav")

start_trim = detect_leading_silence(sound)
end_trim = detect_leading_silence(sound.reverse())

duration = len(sound)    
trimmed_sound = sound[start_trim:duration-end_trim]

In [33]:
trimmed_sound